In [1]:
import sys
import os

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from django.core import serializers

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


### Parsing MDB data

In [2]:
from mdb_stammdata_parser import *

In [9]:
#pm.Person.objects.filter(information_source="MDB Stammdata").all().delete()

(14405,
 {'parliament.Interjection_persons': 0,
  'parliament.ListMembership': 0,
  'parliament.Seat': 7800,
  'parliament.ConstituencyVote1': 0,
  'parliament.Post': 0,
  'parliament.Person': 6605})

In [8]:
#pm.Utterance.objects.filter(speaker__active_country=cmodels.Country.objects.get(name="Germany").id).delete()

(8994,
 {'scoping.Note': 0,
  'scoping.DocOwnership': 0,
  'tmv_app.DocTopic': 0,
  'parliament.Utterance_search_matches': 0,
  'parliament.Paragraph_search_matches': 0,
  'parliament.Interjection_parties': 2626,
  'parliament.Interjection_persons': 916,
  'parliament.Interjection': 2586,
  'parliament.Paragraph': 2423,
  'parliament.Utterance': 443})

In [3]:
parse_mdb_data()

read data from ./mdbs/MDB_STAMMDATEN.XML
<Element 'DOCUMENT' at 0x7f2b1677ea98>
Done. 24 warnings.


### Adding positions

**Positions added manually to spreadsheet, then added to database here**

In [138]:
import pandas as pd 
pos_add = pd.read_csv('mdb_positions_added.csv')
pos_add

,surname,first_name,position added
0,Klein,Hans,Vizepräsident
1,Wimmer,Willy,Parl. Staatssekretär
2,Schmidt,Renate,Vizepräsidentin
3,Schäfer,Helmut,Staatsminister
4,Klein,Hans,Bundesminister für wirtschaftliche Zusammenarbeit
5,Schäfer,Helmut,Staatsminister im Auswärtigen Amt
6,Geiger,Michaela,Parl. Staatssekretär beim Bundesminister für w...
7,Genscher,Hans-Dietrich,Bundesminister Auswärtigen
8,Töpfer,Klaus,"Bundesminister für Umwelt, Naturschutz und Rea..."
9,Töpfer,Klaus,Bundesminister


In [139]:
# will continue to add same position if repeated! 
# 0 - 73 added (server)
for i in range(93, len(pos_add)):
    try:
        surname = pos_add.iloc[i]['surname']
        first_name = pos_add.iloc[i]['first_name']
        position = pos_add.iloc[i]['position added']
        person = pm.Person.objects.get(surname=surname, first_name=first_name)
        person.positions.append(position)
        person.save()
    except MultipleObjectsReturned:
        print(surname, first_name)

In [84]:
# add jäger as alt surname to richard jaeger 
jaeger = pm.Person.objects.get(surname='Jaeger', first_name='Richard')
jaeger.alt_surnames.append('Jäger')
jaeger.positions.append('Vizepräsident')
jaeger.save()

In [88]:
# fix ernst albrecht once
albrecht = pm.Person.objects.get(surname='Albrecht', first_name='Ernst')
albrecht.positions = ['parliamentarian']
albrecht.save()
albrecht.positions

['parliamentarian']

In [118]:
# adding positions as there is more than one karl weber 
weber = pm.Person.objects.filter(surname='Weber', first_name='Karl', in_parlperiod__contains=['4'])[0]
weber.positions.append('Berichterstatter')
weber.save()

In [127]:
# adding positions as there is more than one karl müller 
muller = pm.Person.objects.get(surname='Müller', first_name = 'Karl', in_parlperiod__contains=['1'])
muller.positions.append('Berichterstatter')
muller.save()

In [137]:
hmuller = pm.Person.objects.get(surname='Müller', first_name = 'Heinrich', in_parlperiod__contains=['1'])
hmuller.positions.append('Berichterstatter')
hmuller.save()

In [107]:
# add gerstenmaler as alt surname to eugen gerstenmaier 
gerstenmaier = pm.Person.objects.get(surname='Gerstenmaier', first_name='Eugen')
gerstenmaier.alt_surnames.append('Gerstenmaler')
gerstenmaier.save()

### Adding new people

In [140]:
pos2_add = pd.read_csv('nonelect_positions_added.csv')
pos2_add

,surname,first_name,wp,position added,ortszusatz,title,party
0,Albrecht,Ernst,"11, 10, 9, 8",Ministerpräsident,NaN,Dr.,CDU
1,Franke,Klaus,10,Senator,Berlin,NaN,CDU
2,Vogel,Bernhard,10,Ministerpräsident,Rheinland-Pfalz,NaN,CDU
3,Rau,Johannes,"13, 12, 11, 10, 9, 8",Ministerpräsident,Nordrhein-Westfalen,Dr. h. c.,SPD
4,Becker,Kurt,9,"Staatssekretär, Chef Presseund Informationsamt...",NaN,NaN,NaN
5,Becker,Kurt,9,Staatssekretär,NaN,NaN,NaN
6,Becker,Kurt,9,Staatssekretär im Bundeskanzleramt,NaN,NaN,NaN
7,Schmid,Albert,"9, 8",Staatssekretär im Bundesministerium für Raumor...,NaN,Dr.,SPD
8,Schmid,Albert,"9, 8",Staatssekretär,NaN,NaN,SPD
9,Lange,Volker,9,Senator,Hamburg,NaN,SPD


In [155]:
pos2_add.loc[0]['wp']

'11, 10, 9, 8'

In [176]:
# ernst albrecht (hamburg) already exists, but not the right person
# so won't work with get_or_create
person = pm.Person(surname='Albrecht', first_name='Ernst')
position = 'Ministerpräsident'
title = 'Dr.'
party = 'CDU'
wp = pos2_add.loc[0]['wp']
person.positions = [position]
person.title = title
person.in_parlperiod = []
for i in wp.split(','): 
        person.in_parlperiod.append(i)
person.save()

In [184]:
person = pm.Person(surname='Franke', first_name='Klaus')
position = 'Senator'
ortszusatz = 'Berlin'
party = 'CDU'
wp = pos2_add.loc[1]['wp']
person.positions = [position]
person.ortszusatz = ortszusatz
person.in_parlperiod = []
for i in wp.split(','): 
        person.in_parlperiod.append(i)
person.save()

In [179]:
from django.core.exceptions import MultipleObjectsReturned
for i in range(2, len(pos2_add)):
    surname = pos2_add.iloc[i]['surname']
    first_name = pos2_add.iloc[i]['first_name']
    position = pos2_add.iloc[i]['position added']
    title = pos2_add.iloc[i]['title']
    party = pos2_add.iloc[i]['party']
    ortszusatz = pos2_add.iloc[i]['ortszusatz']
    wp = pos2_add.iloc[i]['wp']
    try:    
        person, created = pm.Person.objects.get_or_create(surname=surname, first_name=first_name)
        if created:
            person.in_parlperiod = []
            person.positions = []
            person.positions = [position]
            for i in wp.split(','): 
                person.in_parlperiod.append(i)   
            if title:
                person.title = title
            if party:
                try:
                    party_obj = pm.Party.objects.get(alt_names__contains=[party])
                    person.party = party_obj
                except pm.Party.DoesNotExist:
                    print("! Warning: party could not be identified: {}".format(party))
            if ortszusatz:
                person.ortszusatz = ortszusatz     
        else:
            person.positions.append(position)
            for i in wp.split(','): 
                person.in_parlperiod.append(i)
        person.save()
    except MultipleObjectsReturned:
        print(surname, first_name)

! Warning: party could not be identified: nan


### Testing

In [183]:
pm.Person.objects.get(surname='Vogel', first_name = 'Bernhard').positions

['parliamentarian', 'Ministerpräsident']

In [177]:
pm.Person.objects.filter(surname='Albrecht', first_name = 'Ernst')

<QuerySet [<Person: Ernst Albrecht (Hamburg)>, <Person: Dr. Ernst Albrecht>]>

In [100]:
pm.Party.objects.get(name='cducsu').alt_names

['CDU',
 'CSU',
 'CDU/CSU',
 'Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union',
 'Fraktion der CDU/CSU (Gast)',
 'CSUS',
 'DSU']

In [106]:
pm.Party.objects.filter(alt_names__contains=['CSU'])

<QuerySet [<Party: CDUCSU>]>